In [1]:
cd ../..

/code


In [3]:
%run "source/config/notebook_settings.py"
import os
import mlflow
from mlflow.tracking import MlflowClient
from helpsk.utility import read_pickle
import helpsk as hlp

from source.library.utilities import Timer, log_info, get_config

config = get_config()
mlflow_uri = config['MLFLOW']['URI']
log_info(f"MLFlow URI: {mlflow_uri}")

client = MlflowClient(tracking_uri=mlflow_uri)

2022-06-12 16:02:09 - INFO     | MLFlow URI: http://mlflow_server:1235


# Get Latest Experiment Run from MLFlow

In [4]:
# Get the production model version and actual model
production_model_info = client.get_latest_versions(name=config['MLFLOW']['MODEL_NAME'], stages=['Production'])
assert len(production_model_info) == 1
production_model_info = production_model_info[0]
production_model = read_pickle(client.download_artifacts(
    run_id=production_model_info.run_id,
    path='model/model.pkl'
))
log_info(f"Production Model Version: {production_model_info.version}")

2022-06-12 16:02:13 - INFO     | Production Model Version: 2


In [5]:
# get experiment and latest run info
credit_experiment = client.get_experiment_by_name(name=config['MLFLOW']['EXPERIMENT_NAME'])
runs = client.list_run_infos(experiment_id=credit_experiment.experiment_id)
latest_run = runs[np.argmax([x.start_time for x in runs])]

In [6]:
yaml_path = client.download_artifacts(run_id=latest_run.run_id, path='experiment.yaml')
results = hlp.sklearn_eval.MLExperimentResults.from_yaml_file(yaml_file_name = yaml_path)

In [7]:
# get the best estimator from the BayesSearchCV
best_estimator = read_pickle(client.download_artifacts(
    run_id=latest_run.run_id,
    path='model/model.pkl'
))

In [8]:
best_estimator.model

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   TransformerChooser(transformer=SimpleImputer(strategy='median'))),
                                                                  ('scaler',
                                                                   TransformerChooser()),
                                                                  ('pca',
                                                                   TransformerChooser(transformer=PCA(n_components='mle')))]),
                                                  ['duration', 'credit_amount',
                                                   'installment_commitment',
                                                   'residence_since', 'age',
                                                   'existing_credi...
                                                   'employment',
                                                   'personal_status',
                                                   'other_parties',
                                                   'property_magnitude',
                                                   'other_payment_plans',
                                                   'housing', 'job',
                                                   'own_telephone',
                                                   'foreign_worker'])])),
                ('model',
                 RandomForestClassifier(criterion='entropy', max_depth=99,
                                        max_features=0.031837350792579364,
                                        max_samples=0.9248344222191298,
                                        min_samples_leaf=4,
                                        min_samples_split=16, n_estimators=1235,
                                        random_state=42))])

# Training & Test Data Info

In [9]:
client.download_artifacts(run_id=latest_run.run_id, path='x_train.pkl')

'/code/mlflow-artifact-root/1/e9af36e49d6c45edbcefca98c6921560/artifacts/x_train.pkl'

In [10]:
with Timer("Loading training/test datasets"):
    X_train = pd.pandas.read_pickle(client.download_artifacts(run_id=latest_run.run_id, path='x_train.pkl'))
    X_test = pd.pandas.read_pickle(client.download_artifacts(run_id=latest_run.run_id, path='x_test.pkl'))
    y_train = pd.pandas.read_pickle(client.download_artifacts(run_id=latest_run.run_id, path='y_train.pkl'))
    y_test = pd.pandas.read_pickle(client.download_artifacts(run_id=latest_run.run_id, path='y_test.pkl'))

2022-06-12 16:03:08 - INFO     | *****Timer Started: Loading training/test datasets
2022-06-12 16:03:08 - INFO     | *****Timer Finished (0.10 seconds)


In [11]:
log_info(X_train.shape)
log_info(len(y_train))

log_info(X_test.shape)
log_info(len(y_test))

2022-06-12 16:03:08 - INFO     | (800, 20)
2022-06-12 16:03:08 - INFO     | 800
2022-06-12 16:03:08 - INFO     | (200, 20)
2022-06-12 16:03:08 - INFO     | 200


In [12]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([559, 241]))

In [13]:
np.unique(y_train, return_counts=True)[1] / np.sum(np.unique(y_train, return_counts=True)[1])

array([0.69875, 0.30125])

In [14]:
np.unique(y_test, return_counts=True)[1] / np.sum(np.unique(y_test, return_counts=True)[1])

array([0.705, 0.295])

# Cross Validation Results

## Best Scores/Params

In [ ]:
log_info(f"Best Score: {results.best_score}")

In [ ]:
log_info(f"Best Params: {results.best_params}")

In [ ]:
# Best model from each model-type.
df = results.to_formatted_dataframe(return_style=False, include_rank=True)
df["model_rank"] = df.groupby("model")["roc_auc Mean"].rank(method="first", ascending=False)
df.query('model_rank == 1')

In [ ]:
results.to_formatted_dataframe(return_style=True,
                               include_rank=True,
                               num_rows=500)

In [ ]:
results.to_formatted_dataframe(query='model == "RandomForestClassifier()"', include_rank=True)

In [ ]:
results.to_formatted_dataframe(query='model == "LogisticRegression()"', include_rank=True)

## BayesSearchCV Performance Over Time

In [ ]:
results.plot_performance_across_trials(facet_by='model').show()

In [ ]:
results.plot_performance_across_trials(query='model == "RandomForestClassifier()"').show()

---

## Variable Performance Over Time

In [ ]:
results.plot_parameter_values_across_trials(query='model == "RandomForestClassifier()"').show()

---

## Scatter Matrix

In [ ]:
# results.plot_scatter_matrix(query='model == "RandomForestClassifier()"',
#                             height=1000, width=1000).show()

---

## Variable Performance - Numeric

In [ ]:
results.plot_performance_numeric_params(query='model == "RandomForestClassifier()"',
                                        height=800)

In [ ]:
results.plot_parallel_coordinates(query='model == "RandomForestClassifier()"').show()

---

## Variable Performance - Non-Numeric

In [ ]:
results.plot_performance_non_numeric_params(query='model == "RandomForestClassifier()"').show()

---

In [ ]:
results.plot_score_vs_parameter(
    query='model == "RandomForestClassifier()"',
    parameter='max_features',
    size='max_depth',
    color='encoder',
)

---

In [ ]:
# results.plot_parameter_vs_parameter(
#     query='model == "XGBClassifier()"',
#     parameter_x='colsample_bytree',
#     parameter_y='learning_rate',
#     size='max_depth'
# )

In [ ]:
# results.plot_parameter_vs_parameter(
#     query='model == "XGBClassifier()"',
#     parameter_x='colsample_bytree',
#     parameter_y='learning_rate',
#     size='imputer'
# )

---

# Best Model - Test Set Performance

In [ ]:
test_predictions = best_estimator.predict(X_test)
test_predictions[0:10]

In [ ]:
evaluator = hlp.sklearn_eval.TwoClassEvaluator(
    actual_values=y_test,
    predicted_scores=test_predictions,
    score_threshold=0.37
)

In [ ]:
evaluator.plot_actual_vs_predict_histogram()

In [ ]:
evaluator.plot_confusion_matrix()

In [ ]:
evaluator.all_metrics_df(return_style=True,
                         dummy_classifier_strategy=['prior', 'constant'],
                         round_by=3)

In [ ]:
evaluator.plot_roc_auc_curve().show()

In [ ]:
evaluator.plot_precision_recall_auc_curve().show()

In [ ]:
evaluator.plot_threshold_curves(score_threshold_range=(0.1, 0.7)).show()

In [ ]:
evaluator.plot_precision_recall_tradeoff(score_threshold_range=(0.1, 0.6)).show()

In [ ]:
evaluator.calculate_lift_gain(return_style=True)

---

# Production Model - Test Set Performance

In [ ]:
test_predictions = production_model.predict(X_test)
test_predictions[0:10]

In [ ]:
evaluator = hlp.sklearn_eval.TwoClassEvaluator(
    actual_values=y_test,
    predicted_scores=test_predictions,
    score_threshold=0.37
)

In [ ]:
evaluator.plot_actual_vs_predict_histogram()

In [ ]:
evaluator.plot_confusion_matrix()

In [ ]:
evaluator.all_metrics_df(return_style=True,
                         dummy_classifier_strategy=['prior', 'constant'],
                         round_by=3)

In [ ]:
evaluator.plot_roc_auc_curve().show()

In [ ]:
evaluator.plot_precision_recall_auc_curve().show()

In [ ]:
evaluator.plot_threshold_curves(score_threshold_range=(0.1, 0.7)).show()

In [ ]:
evaluator.plot_precision_recall_tradeoff(score_threshold_range=(0.1, 0.6)).show()

In [ ]:
evaluator.calculate_lift_gain(return_style=True)

---